In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
#Load Data from SQLite File
import sqlite3
import pandas as pd

In [43]:
# Specify the path to SQLite file in Google Drive
sqlite_file_path = '/content/drive/MyDrive/password_data.sqlite'

In [85]:
# Connect to the SQLite database
conn = sqlite3.connect(sqlite_file_path)

In [86]:
# Load data into a pandas DataFrame
query = "SELECT password, strength FROM Users"
data = pd.read_sql_query(query, conn)

In [87]:
# Close connection
conn.close()

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [89]:
# Extract passwords and labels
X = data['password']
y = data['strength']

In [90]:
# Convert passwords to numeric features with TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='char', max_features=100)  # Ajustez max_features selon vos besoins
X_tfidf = vectorizer.fit_transform(X).toarray()


In [91]:
# Split data into training, validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)



In [92]:
# Calculate class weights
from sklearn.utils import class_weight
import numpy as np

In [93]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


In [94]:
# Build and Train the Neural Network with class weights
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [95]:
# Compile the model
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [96]:
# Train the model with class weights and early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), class_weight=class_weights_dict, callbacks=[early_stopping])


Epoch 1/50
938/938 [==============================] - 5s 4ms/step - loss: 0.8614 - accuracy: 0.5306 - val_loss: 0.7110 - val_accuracy: 0.6511
Epoch 2/50
938/938 [==============================] - 8s 9ms/step - loss: 0.6014 - accuracy: 0.6228 - val_loss: 0.6811 - val_accuracy: 0.6554
Epoch 3/50
938/938 [==============================] - 5s 6ms/step - loss: 0.5338 - accuracy: 0.6569 - val_loss: 0.6273 - val_accuracy: 0.6779
Epoch 4/50
938/938 [==============================] - 5s 6ms/step - loss: 0.4950 - accuracy: 0.6780 - val_loss: 0.5886 - val_accuracy: 0.7027
Epoch 5/50
938/938 [==============================] - 4s 4ms/step - loss: 0.4705 - accuracy: 0.6922 - val_loss: 0.5631 - val_accuracy: 0.7211
Epoch 6/50
938/938 [==============================] - 4s 4ms/step - loss: 0.4504 - accuracy: 0.7037 - val_loss: 0.5433 - val_accuracy: 0.7255
Epoch 7/50
938/938 [==============================] - 5s 5ms/step - loss: 0.4393 - accuracy: 0.7120 - val_loss: 0.5116 - val_accuracy: 0.7376
Epoch 

In [97]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')


625/625 [==============================] - 1s 1ms/step - loss: 0.4357 - accuracy: 0.7883
Test Accuracy: 0.79


In [98]:
# Make predictions on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = y_test_pred.argmax(axis=1)

625/625 [==============================] - 1s 1ms/step


In [99]:
# View classification report for test set
print(classification_report(y_test, y_test_pred_classes, target_names=['Weak', 'Medium', 'Strong']))


              precision    recall  f1-score   support

        Weak       0.46      0.98      0.63      2700
      Medium       0.97      0.74      0.84     14852
      Strong       0.71      0.90      0.79      2448

    accuracy                           0.79     20000
   macro avg       0.72      0.87      0.75     20000
weighted avg       0.87      0.79      0.80     20000



In [100]:
# Function to predict the strength of a password
def predict_password_strength(password):
    password_transformed = vectorizer.transform([password]).toarray()
    strength_pred = model.predict(password_transformed)
    strength_class = strength_pred.argmax(axis=1)[0]
    return ['Weak', 'Medium', 'Strong'][strength_class]

In [110]:
# Prediction example
new_password = ""
predicted_strength = predict_password_strength(new_password)
print(f'The predicted strength of the password "{new_password}" is: {predicted_strength}')

1/1 [==============================] - 0s 34ms/step
The predicted strength of the password "hhhhhhhhhhhccGGG_@FSJSK52424hhhhhhhhhhhhhhhhhhhhhhhhhhhhh" is: Weak


In [102]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [103]:
# Split data into training, validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)


In [104]:
# Initialize and train the RandomForestClassifier model
rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [105]:
# Make predictions on the test set
y_test_pred_rf = rf_model.predict(X_test)

In [106]:
# View classification report for test set
print(classification_report(y_test, y_test_pred_rf, target_names=['Weak', 'Medium', 'Strong']))

              precision    recall  f1-score   support

        Weak       0.94      0.62      0.75      2700
      Medium       0.92      0.99      0.95     14852
      Strong       0.96      0.86      0.91      2448

    accuracy                           0.92     20000
   macro avg       0.94      0.83      0.87     20000
weighted avg       0.93      0.92      0.92     20000



In [107]:
# Function to predict password strength with RandomForestClassifier
def predict_password_strength_rf(password):
    password_transformed = vectorizer.transform([password]).toarray()
    strength_pred = rf_model.predict(password_transformed)[0]
    return ['Weak', 'Medium', 'Strong'][strength_pred]

In [111]:
# Example of prediction with RandomForestClassifier
new_password = "charif"
predicted_strength_rf = predict_password_strength_rf(new_password)
print(f'The predicted strength of the password "{new_password}" is: {predicted_strength_rf}')

The predicted strength of the password "charif" is: Weak
